# AIPI 590 - XAI | Assignment #10
### Hongxuan Li

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1co5boaEJGPXlZuJIFlYYS4KrGH3o6mz0?usp=sharing)

## References

- imodels: https://github.com/csinva/imodels
- sklearn: https://scikit-learn.org/stable/

# Dependencies

In [5]:
import re
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_classification
from sklearn.tree import plot_tree
from imodels import RuleFitClassifier, BoostedRulesClassifier, HSTreeClassifierCV

warnings.filterwarnings('ignore')
seed = 42

# RuleFit Model Explanations

![Alt text](../assignment10/rulefit.png)

# Pipeline of RuleFit based on IMODEL Repo


### Step 1: Tree Generation
**Objective**: Generate a tree ensemble using gradient boosting. This ensemble will be used for for rule extraction.

**Pseudocode**:
```plaintext
def generate_trees(data, parameters):
    ensemble = gradient_boosting(data, parameters)
    return ensemble
```



### Step 2: Rule Extraction
**Objective**: Extract rules from the ensemble of weak tree classifiers. Each rule corresponds to a path from the root to a terminal node in the tree, defined by the splits at each node.

**Pseudocode**:
```plaintext
def extract_rules(tree_ensemble):
    rules = []
    for each tree in tree_ensemble:
        for each node in tree:
            if node is terminal:
                rule = trace_path_to_root(node)
                rules.append(rule)
    return rules
```


### Step 3: Combine Rules and Original Features
**Objective**: Each rule is supplemented to the dataset to create a new binary feature that indicates whether the data point follows the rule.

**Pseudocode**:
```plaintext
def add_features(data, rules):
    transformed_data = []
    for each rule in rules:
        binary_feature = apply_rule(data, rule)
        transformed_data.append(binary_feature)
    return concatenate(data, transformed_data)
```



### Step 4: Linear Modeling
**Objective**: Fit a LASSO using the original features and the newly created binary features from the rules. This model will estimate the feature importance, including both original and rule-based features, on the output target.

**Pseudocode**:
```plaintext
def fit_lasso_model(data, target):
    model = lasso_regression(data, target)
    return model
```


### Step 5: Prediction
**Objective**: Use the Lasso model to make predictions. The model uses the coefficients estimated for both the original and rule-based features to predict the output.

**Pseudocode**:
```plaintext
def predict(model, test_data):
    prediction = model.predict(test_data)
    return prediction
```


# IMODEL Example

In [6]:
# Load data
column_names = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num',
    'marital-status', 'occupation', 'relationship', 'race', 'sex',
    'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income'
]

# Load the training data
train_data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
train_df = pd.read_csv(train_data_url, names=column_names, sep=', ', engine='python', na_values='?')

# Load the test data
test_data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test"
test_df = pd.read_csv(test_data_url, names=column_names, sep=', ', engine='python', skiprows=1, na_values='?')

# Remove leading space
test_df['income'] = test_df['income'].str.strip()

# Convert income labels to 0 and 1
train_df['income'] = train_df['income'].map({'>50K': 1, '<=50K': 0})
test_df['income'] = test_df['income'].map({'>50K': 1, '<=50K': 0})

# get features and target
X = train_df.drop('income', axis=1)
y = train_df['income']


# convert categorical variables to numerical variables
categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
label_encoders = {}  # Dictionary to store encoders for each column
for col in categorical_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le  # Store the fitted LabelEncoder for each column 

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

In [7]:
# Train the model
model = RuleFitClassifier()
model.fit(X_train, y_train)

RuleFitClassifier()

In [8]:
# Evaluate the model
y_pred = model.predict(X_test)
auc = roc_auc_score(y_test.values, y_pred)
print(f'Mean Squared Error: {auc:.2f}')

Mean Squared Error: 0.72


In [9]:
# get rule importances
rule_df = model.visualize()
rule_df["abs_coef"] = rule_df["coef"].abs()
rule_df = rule_df.sort_values(by='abs_coef', ascending=False)
rule_df.iloc[:, [0, 1]]

,rule,coef
20,capital-gain <= 7073.5 and capital-loss <= 2408.5 and relationship <= 4.5 and relationship > 0.5,-1.39
27,capital-gain <= 5095.5 and capital-loss <= 1794.5,-1.37
24,capital-gain <= 5095.5 and hours-per-week <= 43.5,-0.43
18,capital-gain <= 7731.5 and relationship <= 4.5 and relationship > 0.5,-0.41
15,age <= 33.5 and capital-gain <= 6376.5,-0.40
16,capital-gain <= 7073.5 and education-num <= 13.5 and marital-status > 2.5,-0.39
19,age <= 35.5,-0.32
29,capital-gain <= 7073.5,-0.31
35,age > 29.5 and workclass <= 4.5 and education-num > 9.5 and relationship <= 0.5,0.29
14,capital-gain <= 7268.5 and education-num <= 13.5 and relationship <= 4.5 and relationship > 0.5,-0.26


In [10]:
# interpret rules
def interpret_rules(df, label_encoders):
    interpretations = []
    
    for _, row in df.iterrows():
        rule = row['rule']
        coef = row['coef']
        
        # Extract column names and conditions
        conditions = re.findall(r'(\w+(?:-\w+)*)\s*([<>=]+)\s*([\d.]+)', rule)
        
        interpretation = f"Rule {row.name}: "
        for col, op, value in conditions:
            if col in label_encoders:
                # Handle categorical variables
                encoded_value = float(value)
                if col == 'relationship':
                    # Special handling for relationship
                    if op == '<=' and encoded_value > 1:
                        interpretation += f"{col} is one of {label_encoders[col].inverse_transform(range(int(encoded_value)))}; "
                    elif op == '>' and encoded_value < 1:
                        interpretation += f"{col} is one of {label_encoders[col].inverse_transform(range(int(encoded_value)+1, len(label_encoders[col].classes_)))}; "
                else:
                    # For other categorical variables
                    original_value = label_encoders[col].inverse_transform([int(float(value))])[0]
                    interpretation += f"{col} is {original_value}; "
            else:
                # Handle numerical variables
                interpretation += f"{col} {op} {value}; "
        
        interpretation += f"Impact: {coef}"
        interpretations.append(interpretation)
    
    return interpretations
interpretations = interpret_rules(rule_df, label_encoders)
for interp in interpretations:
    print(interp)

Rule 20: capital-gain <= 7073.5; capital-loss <= 2408.5; relationship is one of ['Husband' 'Not-in-family' 'Other-relative' 'Own-child']; relationship is one of ['Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']; Impact: -1.39
Rule 27: capital-gain <= 5095.5; capital-loss <= 1794.5; Impact: -1.37
Rule 24: capital-gain <= 5095.5; hours-per-week <= 43.5; Impact: -0.43
Rule 18: capital-gain <= 7731.5; relationship is one of ['Husband' 'Not-in-family' 'Other-relative' 'Own-child']; relationship is one of ['Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']; Impact: -0.41
Rule 15: age <= 33.5; capital-gain <= 6376.5; Impact: -0.4
Rule 16: capital-gain <= 7073.5; education-num <= 13.5; marital-status is Married-civ-spouse; Impact: -0.39
Rule 19: age <= 35.5; Impact: -0.32
Rule 29: capital-gain <= 7073.5; Impact: -0.31
Rule 35: age > 29.5; workclass is Self-emp-inc; education-num > 9.5; Impact: 0.29
Rule 14: capital-gain <= 7268.5; education-num <= 13.5; relationsh